In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/asl-fingerspelling/supplemental_metadata.csv
/kaggle/input/asl-fingerspelling/character_to_prediction_index.json
/kaggle/input/asl-fingerspelling/train.csv
/kaggle/input/asl-fingerspelling/supplemental_landmarks/371169664.parquet
/kaggle/input/asl-fingerspelling/supplemental_landmarks/369584223.parquet
/kaggle/input/asl-fingerspelling/supplemental_landmarks/1682915129.parquet
/kaggle/input/asl-fingerspelling/supplemental_landmarks/775880548.parquet
/kaggle/input/asl-fingerspelling/supplemental_landmarks/2100073719.parquet
/kaggle/input/asl-fingerspelling/supplemental_landmarks/1650637630.parquet
/kaggle/input/asl-fingerspelling/supplemental_landmarks/1471096258.parquet
/kaggle/input/asl-fingerspelling/supplemental_landmarks/86446671.parquet
/kaggle/input/asl-fingerspelling/supplemental_landmarks/897287709.parquet
/kaggle/input/asl-fingerspelling/supplemental_landmarks/333606065.parquet
/kaggle/input/asl-fingerspelling/supplemental_landmarks/2057261717.parquet
/kaggle/inpu

In [2]:
class Dataset:
    def __init__(self, train_file_path, base_folder_path):
        if base_folder_path.endswith("/"):
            self.base_folder_path = base_folder_path
        else:
            self.base_folder_path = base_foldder_path + "/"
        
        self.train_data = pd.read_csv(train_file_path)
        self.relevant_data = None
        self.pq_path = None
        
    def get_relevant_data_subset(self, pq_path, sequence_id):
        if pq_path == self.pq_path:
            return self.relevant_data.loc[sequence_id]
        
        self.relevant_data = pd.read_parquet(base_folder_path + pq_path)
        self.pq_path = pq_path
        return self.relevant_data.loc[sequence_id]

In [3]:
base_folder_path = "/kaggle/input/asl-fingerspelling/"
train_file_path = "/kaggle/input/asl-fingerspelling/train.csv"
dataset = Dataset(train_file_path, base_folder_path)

In [4]:
def calculate_frame_frequency():    
    total_frames = 0
    total_output_size = 0
    for pq_path, sequence_id, output in zip(dataset.train_data['path'], dataset.train_data['sequence_id'], dataset.train_data['phrase']):
        relevant_data = dataset.get_relevant_data_subset(pq_path, sequence_id)
        total_frames += len(relevant_data)
        total_output_size += len(output)

    print(total_frames)
    print(total_output_size)
    print("Average frames per character: {}".format(total_frames / total_output_size))

In [5]:
from collections import defaultdict, Counter

counter = Counter(''.join(dataset.train_data['phrase']))

for ch, cnt in counter.items():
    print("Character {} appears {} times".format(ch, cnt))
    
counter.most_common(1)[0][0]

Character 3 appears 29789 times
Character   appears 58569 times
Character c appears 33020 times
Character r appears 55533 times
Character e appears 71986 times
Character k appears 12355 times
Character h appears 24243 times
Character o appears 57784 times
Character u appears 20907 times
Character s appears 43224 times
Character a appears 69521 times
Character l appears 38452 times
Character / appears 19477 times
Character y appears 11409 times
Character 1 appears 31292 times
Character 8 appears 28202 times
Character w appears 28532 times
Character i appears 43840 times
Character m appears 26391 times
Character n appears 45663 times
Character 9 appears 29440 times
Character f appears 8639 times
Character 6 appears 29701 times
Character 2 appears 31599 times
Character 0 appears 26744 times
Character t appears 47132 times
Character d appears 25751 times
Character . appears 24652 times
Character j appears 4403 times
Character p appears 17326 times
Character : appears 1939 times
Character z

'e'

In [6]:
# TODO
# 1. Load character map
# 2. Initialize weights so that it always map to most common character
# 3. Build model
# 4. Convert into TFLite
# 5. Submit

In [7]:
import json

with open ("/kaggle/input/asl-fingerspelling/character_to_prediction_index.json", "r") as f:
    character_map = json.load(f)
rev_character_map = {j:i for i,j in character_map.items()}

print(len(rev_character_map))

59


Apparently we are not given a video but rather 3D coordinates of the different points of the body, probably for hiding sensitive information.

In [8]:
import tensorflow as tf
from tensorflow.keras import models, layers

output_size = len(rev_character_map)
input_size = 1630

model = models.Sequential()
layer = layers.Dense(output_size, input_shape=(input_size,))

layer.build((None, input_size))
weights = np.zeros((input_size, output_size))
bias = np.zeros((output_size,))
layer.set_weights([weights, bias])
layer.trainable = False

model.add(layer)
model.compile(optimizer='adam', loss='mse')

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [9]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)